In [1]:
import sys
sys.path.append("../")

import subprocess
import datetime
from joblib import Parallel, delayed

import json
import os

import matplotlib.pyplot as plt
from tqdm import tqdm
import optuna

from run import *

In [ ]:
start_case = 0
case_num = 1000

cases = [
    (f"tools/in/{seed:04}.txt", f"tools/out/{seed:04}.txt")
    for seed in range(start_case, start_case + case_num)
]


def objective(trial: optuna.trial.Trial) -> float:
    min_k = trial.suggest_float("max_k", low=1.5, high=4.5)
    max_k = trial.suggest_float("min_k", low=5.0, high=7.0)
    k_p = trial.suggest_float("k_p", low=0.5, high=2.0)

    start_step = trial.suggest_float("start_step", low=0.4, high=1.2)
    end_step = trial.suggest_float("end_step", low=1.6, high=3.2)
    step_cnt = trial.suggest_int("step_cnt", low=2, high=10)

    args = " ".join(map(str, [min_k, max_k, k_p, start_step, end_step, step_cnt]))

    solver_path = "./target/release/ahc030"
    solver_cmd = f"./tools/target/release/tester {solver_path} {args}"
    solver_version = f"optuna-now-{args}"
    runner = Runner(
        Input,
        Result,
        solver_cmd=solver_cmd,
        solver_version=solver_version,
        database_csv="../log/database.csv",
        log_file="../log/a.log",
        verbose=0,
    )
    runner.run(cases=cases, verbose=0)
    score_df = runner.evaluate_overall_relative_score(ignore_solver_prefix="optuna-now-")
    return score_df.relative_score.median()


subprocess.run("cargo build --features local --release --manifest-path=../Cargo.toml", shell=True)

study = optuna.create_study(
    direction="maximize",
    storage="sqlite:///ahc030.db",
    study_name="tune-steps-01",
    load_if_exists=True,
)
print("start optimize")
study.optimize(objective, n_trials=100, show_progress_bar=True)
study.best_params

In [ ]:
optuna.visualization.plot_slice(study)

In [ ]:
optuna.visualization.plot_optimization_history(study)